# Implementation of a Policy Gradient Model using Keras and a custom Gym environment
Data used in this project can be downloaded from https://www.transtats.bts.gov/Fields.asp. The preprocessing of the data to create the custom gym environment is adapted from this paper https://ieeexplore.ieee.org/document/7411275. 

#### We will use the data from April 2019 in this demonstration

In [5]:
import pandas as pd
from datetime import datetime, date, time, timedelta
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('data.csv').drop('Unnamed: 24', axis=1)
df = df[(df.CANCELLED == 0) & (df.DIVERTED == 0)].drop(
    ['CANCELLED', 'DIVERTED'], axis=1)

# Data Processing
X1 is going to be Taxi Out time, our target

In [2]:
df['x1'] = df['TAXI_OUT']

## Windowing
To be able to achieve the windowing described in the paper we will need to convert DEP_TIME and ARR_TIME into a python datetime object. We will also use the date from FL_DATE

In [3]:
def time_converter(in_):
    in_date = in_[0]
    in_time = in_[1]
    in_time = str(in_time)
    if '.' in in_time:
        in_time = in_time[:-2]
    while len(str(in_time)) < 4:
        in_time = '0' + in_time
    if in_time[:2] == '24':
        in_time = '00' + in_time[2:]
    time_ = time(int(in_time[:2]), int(in_time[2:4]), 0)
    in_date = in_date.split('-')
    date_ = date(int(in_date[0]), int(in_date[1]), int(in_date[2]))
    return datetime.combine(date_, time_)


time_cols = ['CRS_DEP_TIME', 'DEP_TIME', 'WHEELS_OFF',
             'WHEELS_ON', 'CRS_ARR_TIME', 'ARR_TIME', ]

for i, col in enumerate(time_cols):
    df[col] = df[['FL_DATE', col]].apply(time_converter, axis=1)
    print(f"{round(100*(i+1)/len(time_cols),2)}% done.")

16.67% done.
33.33% done.
50.0% done.
66.67% done.
83.33% done.
100.0% done.


In [4]:
def get_bounds(in_time):
    time_d = timedelta(minutes=20)
    return (in_time - time_d),  (in_time + time_d)


def window_func(TIME, variable, df):
    lower_bound, upper_bound = get_bounds(TIME)
    df_ = df[(df[variable] >= lower_bound) & (df[variable] <=
                                              upper_bound)].copy()
    count_airplanes = len(df_) - 1
    return count_airplanes


def pct_tracker(to_be_done):
    done = 0
    count = 0
    while True:
        yield
        done += 1
        count += 1
        if done > to_be_done:
            done -= to_be_done
        if (count >= to_be_done * 0.02):
            count = 0
            pct = round((float(done) / float(to_be_done)) * 100, 2)
            print(f"{pct}% done.")


def track_progress(func=window_func, progress=pct_tracker(len(df))):
    def call_func(*args, **kwargs):
        progress.send(None)
        return func(*args, **kwargs)
    return call_func


def airplane_counter(func, variable, airport_type, list, add_cols=[]):
    listy = []
    sub_df = df.sort_values(by=[airport_type, variable])[
        [airport_type, variable] + add_cols].copy()
    for i, airport in enumerate(list):
        x = sub_df[sub_df[airport_type] == airport].copy()
        x['target'] = x[variable].apply(
            track_progress(func), args=([variable, x]))
        listy.append(x)
    series = pd.concat(listy).sort_values(by=[airport_type, variable]).target
    return series


df = df.sort_values(by=['ORIGIN', 'DEP_TIME']).copy()
origins = df.ORIGIN.unique().tolist()

x2 = airplane_counter(window_func, 'DEP_TIME', 'ORIGIN', origins)

2.0% done.
4.0% done.
6.0% done.
8.0% done.
10.0% done.
12.0% done.
14.0% done.
16.0% done.
18.0% done.
20.0% done.
22.0% done.
24.0% done.
26.0% done.
28.0% done.
30.0% done.
32.0% done.
34.0% done.
36.0% done.
38.0% done.
40.0% done.
42.0% done.
44.0% done.
46.0% done.
48.0% done.
50.0% done.
52.0% done.
54.0% done.
56.0% done.
58.0% done.
60.0% done.
62.0% done.
64.0% done.
66.0% done.
68.0% done.
70.0% done.
72.0% done.
74.0% done.
76.0% done.
78.0% done.
80.0% done.
82.0% done.
84.0% done.
86.0% done.
88.0% done.
90.0% done.
92.0% done.
94.0% done.
96.0% done.
98.0% done.


Similarly, we will calculate the number of arrival aircrafts sharing the runway with the aircraft being considered in the +-20 minutes window (x3)

In [5]:
df['x2'] = x2

df = df.sort_values(by=['DEST', 'ARR_TIME']).copy()
destinations = df.DEST.unique().tolist()

sub_df = df.sort_values(by=['DEST', 'ARR_TIME'])[
    ['DEST', 'ARR_TIME', 'ARR_DELAY']].copy()
dict_ = {dest: sub_df[sub_df['DEST'] == dest].copy() for dest in destinations}


def window_func_arr(in_):
    ORIGIN, TIME = in_[0], in_[1]
    lower_bound, upper_bound = get_bounds(TIME)
    df = dict_[ORIGIN]
    df_ = df[(df['ARR_TIME'] >= lower_bound) & (df['ARR_TIME'] <=
                                                upper_bound)].copy()
    count_airplanes = len(df_)
    return count_airplanes


df['x3'] = df[['ORIGIN', 'DEP_TIME']].apply(
    track_progress(window_func_arr), axis=1)

0.01% done.
2.01% done.
4.01% done.
6.01% done.
8.01% done.
10.01% done.
12.01% done.
14.01% done.
16.01% done.
18.01% done.
20.01% done.
22.01% done.
24.01% done.
26.01% done.
28.01% done.
30.01% done.
32.01% done.
34.01% done.
36.01% done.
38.01% done.
40.01% done.
42.01% done.
44.01% done.
46.01% done.
48.01% done.
50.01% done.
52.01% done.
54.01% done.
56.01% done.
58.01% done.
60.01% done.
62.01% done.
64.01% done.
66.01% done.
68.01% done.
70.01% done.
72.01% done.
74.01% done.
76.01% done.
78.01% done.
80.01% done.
82.01% done.
84.01% done.
86.01% done.
88.01% done.
90.01% done.
92.01% done.
94.01% done.
96.01% done.
98.01% done.


-we will calculate the average taxi out time for each airoprt (x4)

-We will also define a function to check if the flight is or is not during peak time (x5)

-we will take (x6) to be air time and we will take (x7) to be taxi in

In [6]:
def is_peak(dep_time):
    dep_time = dep_time.time()
    if (dep_time >= time(4, 0) and (dep_time <= time(16, 0))):
        return 1
    else:
        return 2


dict_averages={origin: df[df.ORIGIN == origin]
                 ['TAXI_OUT'].mean() for origin in origins}


def avg_taxi_out(origin, dict=dict_averages):
    return dict[origin]


df['x4']=df.ORIGIN.apply(avg_taxi_out)

df['x5']=df.DEP_TIME.apply(is_peak)

df['x6']=df.AIR_TIME

df['x7']=df.TAXI_IN

df.to_csv('snapshot.csv', index=False)

-we will define x* as the number of late aircrafts sharing the runway with the aircraft being considered in the time window

In [7]:
def late_to_leave(TIME, variable, df):
    lower_bound, upper_bound=get_bounds(TIME)
    df_=df[(df[variable] >= lower_bound) & (df[variable] <=
                                              upper_bound) & ((df['DEP_DELAY'] < -15) | (df['DEP_DELAY'] > 15))].copy()
    count_airplanes=len(df_)
    return count_airplanes

x8=airplane_counter(late_to_leave, 'DEP_TIME', 'ORIGIN',
                    origins, add_cols=['DEP_DELAY'])

df['x8']=x8

0.01% done.
2.01% done.
4.01% done.
6.01% done.
8.01% done.
10.01% done.
12.01% done.
14.01% done.
16.01% done.
18.01% done.
20.01% done.
22.01% done.
24.01% done.
26.01% done.
28.01% done.
30.01% done.
32.01% done.
34.01% done.
36.01% done.
38.01% done.
40.01% done.
42.01% done.
44.01% done.
46.01% done.
48.01% done.
50.01% done.
52.01% done.
54.01% done.
56.01% done.
58.01% done.
60.01% done.
62.01% done.
64.01% done.
66.01% done.
68.01% done.
70.01% done.
72.01% done.
74.01% done.
76.01% done.
78.01% done.
80.01% done.
82.01% done.
84.01% done.
86.01% done.
88.01% done.
90.01% done.
92.01% done.
94.01% done.
96.02% done.
98.02% done.


x9 is the number of late arriving aircrafts on the runway in the time window

In [8]:
def late_to_arrive(in_):
    ORIGIN, TIME=in_[0], in_[1]
    lower_bound, upper_bound=get_bounds(TIME)
    df=dict_[ORIGIN]
    df_=df[(df['ARR_TIME'] >= lower_bound) & (df['ARR_TIME'] <=
                                                upper_bound) & ((df['ARR_DELAY'] < -15) | (df['ARR_DELAY'] > 15))].copy()
    count_airplanes=len(df_)
    return count_airplanes

df['x9']=df[['ORIGIN', 'DEP_TIME']].apply(
    track_progress(late_to_arrive), axis=1)

df[['ORIGIN', 'DEP_TIME'] +
    ['x' + str(i) for i in range(1, 10)]].to_csv('variables.csv', index=False)

work_df=df[['ORIGIN', 'DEP_TIME'] +
    ['x' + str(i) for i in range(1, 10)]]

0.02% done.
2.02% done.
4.02% done.
6.02% done.
8.02% done.
10.02% done.
12.02% done.
14.02% done.
16.02% done.
18.02% done.
20.02% done.
22.02% done.
24.02% done.
26.02% done.
28.02% done.
30.02% done.
32.02% done.
34.02% done.
36.02% done.
38.02% done.
40.02% done.
42.02% done.
44.02% done.
46.02% done.
48.02% done.
50.02% done.
52.02% done.
54.02% done.
56.02% done.
58.02% done.
60.02% done.
62.02% done.
64.02% done.
66.02% done.
68.02% done.
70.02% done.
72.02% done.
74.02% done.
76.02% done.
78.02% done.
80.02% done.
82.02% done.
84.02% done.
86.02% done.
88.02% done.
90.02% done.
92.02% done.
94.02% done.
96.02% done.
98.02% done.


Finally we will discretize the target value (taxi out time). our target x1 is going to be descritized into 5 bins

In [2]:
work_df['DEP_TIME']=pd.to_datetime(work_df['DEP_TIME'])
work_df=pd.read_csv('./env/data/variables.csv')
print('non discretized environment variables')
work_df.iloc[:10,:]

non discretized environment variables


,ORIGIN,DEP_TIME,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,SFB,2019-04-01 06:58:00,15.0,7,0,13.801970,1,127.0,4.0,6,0
1,CLT,2019-04-01 09:46:00,27.0,33,28,21.445305,1,68.0,5.0,9,21
2,DTW,2019-04-01 10:09:00,18.0,39,16,17.244655,1,62.0,4.0,13,15
3,ATL,2019-04-01 10:39:00,11.0,58,42,15.761485,1,95.0,3.0,18,22
4,PHL,2019-04-01 12:47:00,14.0,22,9,21.666012,1,15.0,4.0,3,4
5,SFB,2019-04-01 12:13:00,13.0,1,3,13.801970,1,120.0,4.0,1,1
6,ORD,2019-04-01 13:56:00,33.0,65,44,21.773309,1,84.0,5.0,20,20
7,DTW,2019-04-01 15:40:00,21.0,42,22,17.244655,1,61.0,3.0,9,13
8,ATL,2019-04-01 15:49:00,15.0,37,53,15.761485,1,100.0,4.0,10,12
9,CLT,2019-04-01 16:24:00,19.0,52,19,21.445305,2,74.0,6.0,7,10


# Making the environment
We will make a custom gym environment from this data following the code examples in their github.

We will use comments inside the code below to document.

In [1]:
import random
import gym
from gym import spaces
import numpy as np
from datetime import datetime, timedelta

class ontime_dataset_env(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super(ontime_dataset_env, self).__init__()
        self.df = df
        # getting the maximum of each variable because variables have to be bound between 0 and 1
        self.max_x2 = max(df['x2'])
        self.max_x3 = max(df['x3'])
        self.max_x4 = max(df['x4'])
        self.max_x5 = max(df['x5'])
        self.max_x6 = max(df['x6'])
        self.max_x7 = max(df['x7'])
        self.max_x8 = max(df['x8'])
        self.max_x9 = max(df['x9'])
        self.prev_time_step = datetime.strptime(
            '2019-03-30 23:45:01', '%Y-%m-%d %H:%M:%S')
        n_actions = 5
        #defining action space, we are going to discritize our actions into 5 bins
        self.action_space = spaces.Discrete(n_actions)
        
        #defining observation space
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(8,), dtype=np.float16)

    def _next_observation(self):
        # construct the state (observation) for every step, the state will include all the airplains in a window of 15 minutes and we will pad the resulting array with zeros to reach a uniform shape
        obs = np.array([
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x1'].values,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x2'].values / self.max_x2,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x3'].values / self.max_x3,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x4'].values / self.max_x4,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x5'].values / self.max_x5,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x6'].values / self.max_x6,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x7'].values / self.max_x7,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x8'].values / self.max_x8,
            self.df.loc[self.current_step: self.current_step
                        + timedelta(minutes=15), 'x9'].values / self.max_x9
        ])
        #fetch next row until the window is done then get next window
        if self.prev_time_step != self.current_step:
            self.row_idx = 0
            self.input_length = obs.shape[1]
        obs = obs[:, self.row_idx]
        self.prev_time_step = self.current_step
        return obs
    
    def _take_action(self, action):
        # choose a random action
        current_dep_latency = random.randint(0, 4)

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)
        obs = self._next_observation()        
        #if we get to the end of the dataset while taking a step, we return to the begining

        
        #calculating the reward, it will be one for each correct lable and -1* abs(true_label - predicted_label)/4 for each incorrect label
        reward = np.absolute(obs[0] - action)*-1/4
        if reward == 0:
            reward = 1
        self.row_idx += 1
        
        # we will end each episode when we have trained on each row within a 15 minutes window, also if we randomly chose a starting ppoint in the last 15 minutes of the dataset we will be reset to the begining
        done = False if self.row_idx < self.input_length else True
        if done:
            self.current_step = self.current_step = self.df.iloc[random.randint(0, len(self.df.loc[:, 'x2'].values) - 6)].name 
            if self.current_step > datetime.strptime('2019-04-30 23:45:01', '%Y-%m-%d %H:%M:%S'):
                self.current_step = datetime.strptime(
                    '2019-04-01 00:00:00', '%Y-%m-%d %H:%M:%S')
        return obs, reward, done, {}

    def reset(self):
        # Set the current step to a random point within the data frame
        self.current_step = self.df.iloc[random.randint(
            0, len(self.df.loc[:, 'x2'].values) - 6)].name
        return self._next_observation()


    def render(self, mode='human', close=False):
        pass

# Making the Model
Our model is based on stable-baselines PGM model, it's also built on the code available in their github.

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
from tensorflow import keras
from keras.layers import Dense, Activation, Input, Flatten
from keras.models import Model, load_model
from keras.optimizers import Adam
import keras.backend as K
import numpy as np


class Agent(object):
    def __init__(self, ALPHA, GAMMA=0.99, n_actions=5, layer1_size=16, layer2_size=16, layer3_size=16, input_dims=9, fname='model.h5'):
        self.GAMMA = GAMMA
        self.lr = ALPHA
        self.G = 0
        self.input_dims = input_dims
        self.fc1_dims = layer1_size
        self.fc2_dims = layer2_size
        self.fc3_dims = layer3_size
        self.n_actions = n_actions
        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []
        self.policy, self.predict = self.build_policy_network()
        self.action_space = [i for i in range(n_actions)]
        self.model_file = fname
    
    #this is the function that will define the policy agent and and will make predictions
    def build_policy_network(self):
        input = Input(shape=(self.input_dims,))
        advantages = Input(shape=[1])
        dense1 = Dense(self.fc1_dims, activation='relu')(input)
        dense2 = Dense(self.fc2_dims, activation='relu')(dense1)
        dense3 = Dense(self.fc3_dims, activation='relu')(dense2)
        probs = Dense(self.n_actions, activation='softmax')(dense3)

        def custom_loss(y_true, y_predict):
            out = K.clip(y_predict, 1e-8, 1 - 1e-8)
            log_lik = y_true * K.log(out)
            return K.sum(-log_lik * advantages)

        policy = Model(input=[input, advantages], output=[probs])
        policy.compile(optimizer=Adam(lr=self.lr), loss=custom_loss)
        predict = Model(input=[input], output=[probs])
        return policy, predict
    
    #we will choose an actiom for each observation within the batch by making a random choice weighted by the probabilities predicted by our model
    def choose_action(self, observation):
        state = observation[np.newaxis, :]
        probabilities = self.predict.predict(state)[0]
        action = np.random.choice(self.action_space, p=probabilities)
        return action

    #this is a helper function that stores the history of the model
    def store_transition(self, observation, action, reward):
        self.state_memory.append(observation)
        self.action_memory.append(action)
        self.reward_memory.append(reward)
        
    # this is the main driver function that we will call to train the agent
    def learn(self):
        state_memory = np.array(self.state_memory)
        action_memory = np.array(self.action_memory)
        reward_memory = np.array(self.reward_memory)
        actions = np.zeros([len(action_memory), self.n_actions])
        actions[np.arange(len(action_memory)), action_memory] = 1
        
        # calculating the gain
        G = np.zeros_like(reward_memory)
        for t in range(len(reward_memory)):
            G_sum = 0
            discount = 1
            for k in range(t, len(reward_memory)):
                G_sum += reward_memory[k]*discount
                discount *= self.GAMMA
            G[t] = G_sum
        mean = np.mean(G)
        std = np.std(G) if np.std(G) > 0 else 1
        self.G = (G-mean)/std
        
        #calculating the cost
        cost = self.policy.train_on_batch([state_memory, self.G], actions)
        
        #resetting memory
        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []

    
    #helper function to save and load the model
    def save_model(self):
        self.policy.save(self.model_file)

    def load_model(self):
        self.policy = load_model(self.model_file)

Using TensorFlow backend.


# Testing the environment

In [3]:
import gym
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

df = pd.read_csv('./env/data/variables.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal')
df['x1'] = discretizer.fit_transform(
df['x1'].to_numpy().reshape(-1, 1))

env = ontime_dataset_env(df)

In [8]:
env.reset()

array([1.        , 0.03448276, 0.02380952, 0.58825559, 0.5       ,
       0.05890805, 0.0221519 , 0.        , 0.01724138])

In [12]:
env.step(4)

(array([4.        , 0.14942529, 0.70238095, 0.61140748, 0.5       ,
        0.16954023, 0.00949367, 0.1       , 0.15517241]),
 1,
 False,
 {})

In [6]:
env.step(4)

(array([0.        , 0.11494253, 0.27380952, 0.68524502, 1.        ,
        0.27011494, 0.01265823, 0.03333333, 0.13793103]),
 -1.0,
 False,
 {})

Now that we are satisfied that our custom environment is working, we will start training the model.

# Training the model
Unfortunately no matter what hyper parameters were used, this model was not able to be trained on this dataset effectively.

In [7]:
agent = Agent(ALPHA=0.001, GAMMA=0.99, n_actions=5,
                  layer1_size=32, layer2_size=16,
                  layer3_size=8, fname='model.h5')
env = ontime_dataset_env(df)
score_history = []

n_episodes = 2500

for i in range(n_episodes):
    done = False
    score = 0
    observation = env.reset()
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        agent.store_transition(observation, action, reward)
        observation = observation_
        score += reward
    score_history.append(score)
    agent.learn()
    print('episode', i, 'score %.1f' % score, 'average_score %.1f' %
         np.mean(score_history[-100:]))

agent.save_model()

/home/mostafakm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
/home/mostafakm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


episode 0 score -45.2 average_score -45.2
episode 1 score -80.0 average_score -62.6
episode 2 score -71.5 average_score -65.6
episode 3 score -32.5 average_score -57.3
episode 4 score -51.8 average_score -56.2
episode 5 score -74.5 average_score -59.2
episode 6 score -65.8 average_score -60.2
episode 7 score -41.0 average_score -57.8
episode 8 score -85.8 average_score -60.9
episode 9 score -86.5 average_score -63.5
episode 10 score -123.0 average_score -68.9
episode 11 score -65.5 average_score -68.6
episode 12 score -71.5 average_score -68.8
episode 13 score -62.5 average_score -68.4
episode 14 score -88.2 average_score -69.7
episode 15 score -41.0 average_score -67.9
episode 16 score -90.5 average_score -69.2
episode 17 score -46.5 average_score -68.0
episode 18 score -65.2 average_score -67.8
episode 19 score -63.5 average_score -67.6
episode 20 score -60.0 average_score -67.2
episode 21 score -35.8 average_score -65.8
episode 22 score -59.0 average_score -65.5
episode 23 score -80

# Trying the same agent on a gym environment
Below is the same model defined above, just edited to take the new input shapes from Gym's LunarLander-v2 environment. the model was able to be trained on this environment and was able to get positive rewards.

In [1]:
from gym_model.Agent import GymAgent
from gym_model.gym_env import run_model

Using TensorFlow backend.


In [2]:
run_model()

/home/mostafakm/Documents/School/Practical Data Science/Assignment 7/gym_model/Agent.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  policy = Model(input=[input, advantages], output=[probs])
/home/mostafakm/Documents/School/Practical Data Science/Assignment 7/gym_model/Agent.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  predict = Model(input=[input], output=[probs])


episode 0 score -321.2 average_score -321.2
episode 1 score -111.1 average_score -216.1
episode 2 score -157.6 average_score -196.6
episode 3 score -222.1 average_score -203.0
episode 4 score -142.3 average_score -190.8
episode 5 score -291.4 average_score -207.6
episode 6 score -25.4 average_score -181.6
episode 7 score -28.4 average_score -162.4
episode 8 score -261.0 average_score -173.4
episode 9 score -103.2 average_score -166.4
episode 10 score -10.5 average_score -152.2
episode 11 score -153.5 average_score -152.3
episode 12 score -287.2 average_score -162.7
episode 13 score -168.1 average_score -163.1
episode 14 score -300.6 average_score -172.2
episode 15 score -225.9 average_score -175.6
episode 16 score -133.4 average_score -173.1
episode 17 score -141.5 average_score -171.4
episode 18 score -143.3 average_score -169.9
episode 19 score -109.8 average_score -166.9
episode 20 score -149.8 average_score -166.1
episode 21 score -106.6 average_score -163.4
episode 22 score -72.3 

# Conclusion
After assignment 6, this is my second attempt to apply the approach outlined the paper "Reinforcement Learning for Taxi-out Time Prediction: An improved Q-learning Approach, to predict taxi-out time on the on-time dataset.

In assignment 6, i used deep q-learning and in this assignment I tried PGM. both attempts did not yield significant results. I'm lead to believe that this dataset might not be the best candidate for the RL approach.